# Find best parameters describing data, both combined and separated fits. 


In [2]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 
from scipy.optimize import curve_fit
import pickle
import sys
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib as mpl
import scipy.stats as stats
import pandas as pd
from importlib import reload
from scipy.interpolate import interp1d,interp2d

import os
cwd=os.getcwd()

area=4.0;linethicknes=1.0;

font = {'family' : 'normal','weight' : 'normal','size'   : 7, 'family':'serif','serif':['Arial']}
mpl.rc('font', **font)
mpl.rcParams['lines.linewidth'] =linethicknes
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['pdf.fonttype'] = 42
area=4.0;linethicknes=1.0;


########################################################################
############# Import code

sys.path.insert(0, './../../../DataAndScripts/unstructured_scripts/')
import functions_optimal as fun
import sims_utils as su
import functions_dynamics as fun_dyna

# Load data

In [4]:

#######################################
folder_data='./../../../DataAndScripts/experimental_data/'
#######################################

data_monkeys=np.loadtxt(folder_data+'Monkeys_with_trials.txt');
data_mice=np.loadtxt(folder_data+'Mice_with_trials.txt');

data_both_species=[data_mice,data_monkeys]
species=['Mice','Monkeys']

dataset_both_species,Con_both_species,nCon_both_species,normalization_both_species=[],[],[],[]
for idx_species in range(2):
    dataset,Con,nCon=fun.build_dataset(data_both_species[idx_species])
    normalization=1.0;#np.mean(dataset[0,:,0])
    dataset_both_species=dataset_both_species+[dataset]
    Con_both_species=Con_both_species+[Con]
    nCon_both_species=nCon_both_species+[nCon]
    normalization_both_species=normalization_both_species+[normalization]

# Load best fit and simulate

In [3]:
idx_species=1;



dataset=dataset_both_species[idx_species]
Con=Con_both_species[idx_species]
nCon=nCon_both_species[idx_species]
normalization=normalization_both_species[idx_species] 

sol=ALL_sol[idx_species]
cost=ALL_cost[idx_species]
idx_best=np.argmin(cost)
best_param=sol[idx_best,:]
best_cost=cost[idx_best]
best_inputs=fun.fit_inputs_to_data_given_param(dataset,Predictor_data,best_param,nCon)

print(best_param)

NameError: name 'dataset_both_species' is not defined

In [ ]:
sigma_t=0.01; # Input noise, determine how smooth the single neuron transfer function is 
tau_rp=0.002; # Refractory period

g_E= best_param[0]; g_I=best_param[1];beta=10**best_param[2]; # parameters characterizing the relative strength of synaptic connections
K=500; # average number of E connections per neuron (I connections are 1/4 of that)

CV_K=10**best_param[3]
sigma_Lambda_over_Lambda=10**best_param[4]; # ratio between mean and standard deviation of the opsin expression

J=10**best_param[5]; # Synaptic efficacy in Volts
r_X=10**np.asarray(best_inputs[0:-1])
#r_X=10**np.asarray([1.62])
L=[0.,  10.**(best_inputs[-1])]; #  Laser intensity that fits the data

#Tmax_over_tau_E=20;
Tmax_over_tau_E=200;

p=0.1; # probability of connection between cells. 
sigma_X_over_r_X=0.2; # Presynaptic population has mean rate r_X, Gaussianly distributed with std=0.2* r_X
Lambda=1*10**-3; # mean opsin expression in Volt

sigma_Lambda=sigma_Lambda_over_Lambda*Lambda

# In what follows, I compute W_{AB} starting from the parameters defined above
G_E,G_I=1.0,2.0 # Gain of Excitatory and inhibitory cells and I cells
w_EE=1;w_IE=w_EE/beta;
w_EI=g_E*w_EE;w_II=g_I*w_IE;
w_EX,w_IX=(G_I*fun_dyna.gamma*g_E-G_E)*w_EE,(G_I*fun_dyna.gamma*g_I-G_E)*w_IE; 
w_X=np.asarray([w_EX,w_IX]);
w=np.zeros((2,2));
w[0,:]=w_EE,-w_EI
w[1,:]=w_IE,-w_II



In [ ]:
M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I=fun_dyna.Generate_quenched_disorder(CV_K,J,K,w,w_X,p,Lambda,sigma_Lambda)

In [ ]:
'''
mu_tab_max=10.0;
mu_tab=np.linspace(-mu_tab_max,mu_tab_max,200000)
mu_tab=np.concatenate(([-10000],mu_tab))
mu_tab=np.concatenate((mu_tab,[10000]))

phi_tab_E,phi_tab_I=mu_tab*0,mu_tab*0;
for idx in range(len(phi_tab_E)):
    phi_tab_E[idx]=fun_dyna.comp_phi_tab(mu_tab[idx],fun_dyna.tau_E,tau_rp,sigma_t)
    phi_tab_I[idx]=fun_dyna.comp_phi_tab(mu_tab[idx],fun_dyna.tau_I,tau_rp,sigma_t)

phi_int_E=interp1d(mu_tab, phi_tab_E, kind='linear')  
phi_int_I=interp1d(mu_tab, phi_tab_I, kind='linear')
'''


In [ ]:
T=np.arange(0,Tmax_over_tau_E*fun_dyna.tau_E,fun_dyna.tau_I/3);
mask_time=T>(10*fun_dyna.tau_E)
RATES=-1*np.ones((len(r_X),len(L),N_E+N_I))
DYNA=-1*np.ones((len(r_X),len(L),N_E+N_I,len(T)))
MUS=-1*np.ones((len(r_X),len(L),N_E+N_I))
Lambda_i_L=1./tau_rp*np.ones((len(r_X),len(L),N_E+N_I))

print('Computing and saving network response')
for idx_r_X in range(len(r_X)):
    for idx_L in range(len(L)):
        r_X_local,L_local=r_X[idx_r_X],L[idx_L];
        print((idx_r_X+1)/len(r_X),(idx_L+1)/len(L))
        DYNA[idx_r_X,idx_L,:,:], MUS[idx_r_X,idx_L,:],Lambda_i_L[idx_r_X,idx_L,:]=fun_dyna.High_dimensional_dynamics(T,L_local,r_X_local,M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I,phi_int_E,phi_int_I);
        RATES[idx_r_X,idx_L,:]=np.mean(DYNA[idx_r_X,idx_L,:,mask_time],axis=0)
        print(np.mean(RATES[idx_r_X,idx_L,0:N_E]),
              np.mean(RATES[idx_r_X,idx_L,N_E::]),
              np.std(RATES[idx_r_X,idx_L,0:N_E]),
              np.std(RATES[idx_r_X,idx_L,N_E::]))




In [ ]:
bin_edges=np.arange(0,1./tau_rp,.1)
spacings=np.diff(bin_edges)[0];
bin_means=0.5*(bin_edges[0:-1]+bin_edges[1::])
bin_edges_Delta=np.arange(-1./tau_rp,1./tau_rp,.1)
spacings_Delta=np.diff(bin_edges)[0];
bin_means_Delta=0.5*(bin_edges_Delta[0:-1]+bin_edges_Delta[1::])

fig, axs = plt.subplots(len(r_X),4, figsize=(2.5*4,2.0*len(r_X)), facecolor='w', edgecolor='k')#,sharey='row')#,sharex=True)
fig.subplots_adjust(hspace = .5, wspace=.5)
area=1.0;linethicknes=1.0;
axs = axs.ravel()
font = {'family' : 'normal','weight' : 'normal','size'   : 7, 'family':'serif','serif':['Arial']}
mpl.rc('font', **font)
mpl.rcParams['lines.linewidth'] =linethicknes
mpl.rcParams['text.usetex'] = False
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['mathtext.rm'] = 'serif'
mpl.rcParams['pdf.fonttype'] = 42

idx_r_X=0
mask_active_cells=(RATES[idx_r_X,1,:]>=0)|(RATES[idx_r_X,0,:]>=0)
for idx_r_X in np.arange(1,len(r_X),1):
    mask_active_cells=mask_active_cells|(RATES[idx_r_X,1,:]>=0)|(RATES[idx_r_X,0,:]>=0)

for idx_r_X in range(len(r_X)):
    

    color=['k','b',];
    label=['Without opto','With opto']
    for idx_L in range(len(L)):
        #hist, bin_edges=np.histogram(RATES[idx_r_X,idx_L,0:N_E],bin_edges)#,normed=True)
        hist, bin_edges=np.histogram(RATES[idx_r_X,idx_L,mask_active_cells],bin_edges)#,normed=True)
        bin_mean=0.5*(bin_edges[0:-1]+bin_edges[1::])
        spacings=np.diff(bin_edges)[0];
        hist=hist/np.sum(hist*spacings)
        axs[idx_r_X*4+0].scatter(bin_mean,np.cumsum(spacings*hist),c=color[idx_L],s=area,label=label[idx_L])

        
    Delta_Sim=RATES[idx_r_X,1,mask_active_cells]-RATES[idx_r_X,0,mask_active_cells]
    print(len(Delta_Sim[Delta_Sim<0])/len(Delta_Sim))
    
    hist_Delta, bin_edges_Delta=np.histogram(Delta_Sim,bin_edges_Delta,normed=True)
    CDF_Resp_Delta=np.cumsum(hist_Delta*spacings_Delta)
    axs[idx_r_X*4+1].scatter(bin_means_Delta,CDF_Resp_Delta,c='b',s=area);


    #axs[idx_r_X*4+2].scatter(RATES[idx_r_X,0,maks_cells],RATES[idx_r_X,1,maks_cells]-RATES[idx_r_X,0,maks_cells],c='b',s=1,alpha=0.3);
    idx_cells=np.where(np.arange(len(Lambda_i))<N_E)
    axs[idx_r_X*4+2].scatter(RATES[idx_r_X,0,idx_cells],RATES[idx_r_X,1,idx_cells]-RATES[idx_r_X,0,idx_cells],c='r',s=1,alpha=0.1);
    idx_cells=np.where(np.arange(len(Lambda_i))>N_E)
    axs[idx_r_X*4+2].scatter(RATES[idx_r_X,0,idx_cells],RATES[idx_r_X,1,idx_cells]-RATES[idx_r_X,0,idx_cells],c='b',s=1,alpha=0.1);

    cov=np.cov(RATES[idx_r_X,0,mask_active_cells],RATES[idx_r_X,1,mask_active_cells]-RATES[idx_r_X,0,mask_active_cells])
    axs[idx_r_X*4+2].set_title(cov[0,1]/cov[1,1])


    for idx_cell in range(17):
        axs[idx_r_X*4+3].plot(T[mask_time]-T[mask_time][0],DYNA[idx_r_X,1,idx_cell,mask_time],marker='o',markersize=0.1,alpha=0.5)

    axs[idx_r_X*4+0].axis([0,250,.0,1.])
    axs[idx_r_X*4+0].legend(loc='lower right')
    axs[idx_r_X*4+1].axis([-100,100,0,1.])
    axs[idx_r_X*4+0].set_title('J=%.2fmV'%(J*10**3))
    axs[idx_r_X*4+0].set_ylabel('CDF')
    axs[idx_r_X*4+1].set_ylabel('CDF')
    axs[idx_r_X*4+0].set_xlabel('$r$ (spk/s)')
    axs[idx_r_X*4+1].set_xlabel('$\Delta r$ (spk/s)')
    axs[idx_r_X*4+2].set_ylabel('$\Delta r$ (spk/s)')
    axs[idx_r_X*4+2].set_xlabel('$r$ (spk/s)')
    axs[idx_r_X*4+3].set_ylabel('$r$ (spk/s)')
    axs[idx_r_X*4+3].set_xlabel('time (s)')
    axs[idx_r_X*4+3].set_title('Trajectory of example cells')



In [ ]:
moments_of_r_sim=np.zeros((len(r_X),5))
#ATTT
mask_active_cells=(RATES[idx_r_X,1,:]>=0)|(RATES[idx_r_X,0,:]>=0)

'''
idx_r_X=0
mask_active_cells=(RATES[idx_r_X,1,:]>=1)|(RATES[idx_r_X,0,:]>=1)
for idx_r_X in np.arange(1,len(r_X),1):
    mask_active_cells=mask_active_cells|(RATES[idx_r_X,1,:]>=1)|(RATES[idx_r_X,0,:]>=1)
'''
for idx_r_X in range(len(r_X)):
    #mask_active_cells=(RATES[idx_r_X,1,:]>=1)|(RATES[idx_r_X,0,:]>=1)
    Base_Sim=RATES[idx_r_X,0,:][mask_active_cells]
    Delta_Sim=(RATES[idx_r_X,-1,:]-RATES[idx_r_X,0,:])[mask_active_cells]
    
    moments_of_r_sim[idx_r_X,0]=np.mean(Base_Sim)
    moments_of_r_sim[idx_r_X,1]=np.mean(Delta_Sim)
    moments_of_r_sim[idx_r_X,2]=np.std(Base_Sim)
    moments_of_r_sim[idx_r_X,3]=np.std(Delta_Sim)
    pippo=np.cov(Base_Sim,Delta_Sim)
    moments_of_r_sim[idx_r_X,4]=(pippo)[0,1]/(pippo)[1,1]


In [ ]:

rows=4
cols=1

colors=['c','m']
fig, axs = plt.subplots(rows,cols, figsize=(2.5*cols,2.*rows), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()





for idx_moment in range(6):
    if idx_moment<2:
        idx_row=0
        ccc=['k',colors[idx_species]][idx_moment]
        ymin=0
        ymax=[25,80][idx_species]

    if (idx_moment>=2)&(idx_moment<4):
        idx_row=1
        ccc=['k',colors[idx_species]][idx_moment-2]
        ymin=0
        #ymax=[20,60][idx_species]
        ymax=[20,70][idx_species]

    if (idx_moment==4):
        idx_row=2
        ccc=colors[idx_species]
        ymin=0
        ymax=[20,60][idx_species]

    if (idx_moment==5):
        idx_row=3
        ccc=colors[idx_species]
        ymin=-0.75
        ymax=0.25

    m,sem=dataset[idx_moment,:,0],dataset[idx_moment,:,1]
    axs[cols*idx_row].scatter(Con,m,
                                       facecolors='none', s=10,edgecolors=ccc,marker='o'
                                       )
    axs[cols*idx_row].fill_between(Con,m-sem,m+sem,
                                             color=ccc,alpha=0.2)

    axs[cols*idx_row].set_xscale('symlog', linthreshx=12)
    axs[cols*idx_row].axis([-1.,110,ymin,ymax])
axs[cols*idx_row].axhline(y=0,ls='--',c='k')
axs[cols*3].set_xlabel('contrast (%)')
axs[cols*0].set_title(['Mice','Monkeys'][idx_species])
axs[cols*0].set_ylabel('Mean rate\n(spk/s)')
axs[cols*1].set_ylabel('Std rate\n(spk/s)')
axs[cols*2].set_ylabel('Std $\Delta$rate\n(spk/s)')
axs[cols*3].set_ylabel(r'$\rho$')
'''
        
for idx_species in [0,1]:
    print(idx_species)
    dataset=dataset_both_species[idx_species]
    Con=Con_both_species[idx_species]
    nCon=nCon_both_species[idx_species]

    mask_cases=(sim_species==idx_species)&(sim_combined==idx_combined)

    data_x=sim_param_all[mask_cases,12]
    cost=0
    for idx_moment in range(6):
        clr=color[idx_species]

        if idx_moment==0:
            data_y=moments_of_r_sim_all[mask_cases,0]
            idx_plot=2*0+idx_species
            clr='k'
        if idx_moment==1:
            data_y=moments_of_r_sim_all[mask_cases,0]+moments_of_r_sim_all[mask_cases,1]
            idx_plot=2*0+idx_species
        if idx_moment==2:
            data_y=moments_of_r_sim_all[mask_cases,2]
            idx_plot=2*1+idx_species
            clr='k'
        if idx_moment==3:
            data_y=np.sqrt(moments_of_r_sim_all[mask_cases,2]**2
                           +moments_of_r_sim_all[mask_cases,3]**2
                           +2*moments_of_r_sim_all[mask_cases,4])
            idx_plot=2*1+idx_species
        if idx_moment==4:
            data_y=moments_of_r_sim_all[mask_cases,3]
            idx_plot=2*2+idx_species

        if idx_moment==5:
            data_y=moments_of_r_sim_all[mask_cases,4]/moments_of_r_sim_all[mask_cases,3]**2
            idx_plot=2*3+idx_species


        data_y_med=np.zeros((len(np.unique(data_x)),3))
        for idx in range(len(np.unique(data_x))):
            mask_x=data_x==np.unique(data_x)[idx]
            data_y_med[idx,0]=np.mean(data_y[mask_x])
            data_y_med[idx,1]=stats.sem(data_y[mask_x])
        axs[idx_plot].errorbar(Con,data_y_med[:,0],data_y_med[:,1],c=clr,ls=ls) 
        cost_moment=(data_y_med[:,0]-dataset[idx_moment,:,0])/dataset[idx_moment,:,1]
        cost_moment=(0.5*(np.sum(cost_moment**2)))
        cost=cost+cost_moment
        print(cost_moment,cost)
    print('   ',cost)
'''

Prediction=Predictor_data(best_inputs,best_param,nCon,)
for idx_moment in range(5):
    idx_plt=idx_moment
    axs[idx_plt].plot(Con,moments_of_r_sim[:,idx_moment],label='data')



In [ ]:
rows=1
cols=5
colors=['c','m']
fig, axs = plt.subplots(rows,cols, figsize=(2.5*cols,2.*rows), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.5)
axs = axs.ravel()


Prediction=Predictor_data(best_inputs,best_param,nCon,)
for idx_moment in range(5):
    idx_plt=idx_moment
    axs[idx_plt].plot(Con,moments_of_r_sim[:,idx_moment],label='data')


    m,sem=dataset[idx_moment,:,0],dataset[idx_moment,:,1]
    axs[idx_plt].scatter(Con,m,facecolors='none', s=10,edgecolors=ccc,marker='o')
    axs[idx_plt].fill_between(Con,m-sem,m+sem,color=ccc,alpha=0.2)

    axs[idx_plt].plot(Con,Predictions[idx_moment,:],color=ccc,alpha=1.)
    axs[idx_plt].set_xscale('symlog', linthreshx=12)
'''
axs[0].set_ylim([0,20])
axs[1].set_ylim([0,10])
axs[2].set_ylim([0,20])
axs[3].set_ylim([0,20])
axs[4].set_ylim([-10,0])
'''

In [ ]:
np.shape(Prediction)